In [6]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

# Evaluation Abylsen

## Tutoriel

In [18]:
import numpy as np


def cosine(vector1, vector2):
    return (np.linalg.norm(vector1) * np.linalg.norm(vector2)) / np.dot(vector1, vector2)

def euclidean(vector1, vector2):
    return np.sqrt(np.linalg.norm(vector1 - vector2))

In [20]:
import numpy as np


def cosine2(vector1, vector2):
    return (1 - ((np.linalg.norm(vector1) * np.linalg.norm(vector2)) / np.dot(vector1, vector2)))

def euclidean2(vector1, vector2):
    return np.sqrt(np.linalg.norm(vector1 - vector2)**2)

In [4]:
vector1 = np.array([15, 5, 7, 1, 1, 0, 0, 1])
vector2 = np.array([1, 5, 7, 8, 8, 0, 0, 20])

In [19]:
vector1 = np.array([15, 5, 7, 1, 1, 0, 0, 1])
vector2 = np.array([1, 5, 7, 8, 8, 0, 0, 20])

print("cosine: ", cosine(vector1, vector2)) # 0.292919231098
print("euclidean: ", euclidean(vector1, vector2)) # 25.5929677841

cosine:  3.4139103678919276
euclidean:  5.058949276691698


In [21]:
euclidean2(vector1, vector2)

25.592967784139454

In [17]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(vector1, vector2)/(norm(vector1)*norm(vector2))
cos_sim

0.29291923109788465

# Test

In [24]:
folder = 'D:/Data_Analyst_Projet/Projet 8/Donnée debut projet/DAN-P8-donnees/'
pop = pd.read_csv(folder + 'Population.csv')

In [30]:
pop

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779.953
1,Afghanistan,Male,2000,10689.508
2,Afghanistan,Female,2000,10090.449
3,Afghanistan,Rural,2000,15657.474
4,Afghanistan,Urban,2000,4436.282
...,...,...,...,...
20909,Zimbabwe,Total,2018,14438.802
20910,Zimbabwe,Male,2018,6879.119
20911,Zimbabwe,Female,2018,7559.693
20912,Zimbabwe,Rural,2018,11465.748


In [35]:
df = pop[['Year', 'Population']]
df

,Year,Population
0,2000,20779.953
1,2000,10689.508
2,2000,10090.449
3,2000,15657.474
4,2000,4436.282
...,...,...
20909,2018,14438.802
20910,2018,6879.119
20911,2018,7559.693
20912,2018,11465.748


In [38]:
Q1 = df['Population'].quantile(0.25)
Q3 = df['Population'].quantile(0.75)
IQR = Q3 - Q1
IQR

10802.079749999999

In [39]:
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

In [42]:
def handle_outliers(df):
    for col in df :
        print(col)
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Étape 2: Déterminer les limites pour les outliers
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        # Étape 3: Remplacer les valeurs inférieures par la plus petite valeur non-outlier
        min_non_outlier = df[df[col] >= lower_limit][col].min()
        df.loc[df[col] < lower_limit, col] = min_non_outlier

        # Étape 4: Remplacer les valeurs supérieures par la plus grande valeur non-outlier
        max_non_outlier = df[df[col] <= upper_limit][col].max()
        df.loc[df[col] > upper_limit, col] = max_non_outlier

        # Afficher le DataFrame après le remplacement
    return df

In [43]:
handle_outliers(df)

Year
Population


,Year,Population
0,2000,20779.953
1,2000,10689.508
2,2000,10090.449
3,2000,15657.474
4,2000,4436.282
...,...,...
20909,2018,14438.802
20910,2018,6879.119
20911,2018,7559.693
20912,2018,11465.748


In [ ]:
def impute_data(df):
    n_rows, n_cols = df.shape

    def ui(i):
        non_null_row_values = df[i, ~np.isnan(df[i, :])]
        return np.mean(non_null_row_values) if non_null_row_values.size > 0 else np.nan

    def uj(j):
        non_null_col_values = df[:, j][~np.isnan(df[:, j])]
        return np.mean(non_null_col_values) if non_null_col_values.size > 0 else np.nan

    def Mi(i):
        non_null_row_values = df[i, ~np.isnan(df[i, :])]
        return np.median(non_null_row_values) if non_null_row_values.size > 0 else np.nan

    def Mj(j):
        non_null_col_values = df[:, j][~np.isnan(df[:, j])]
        return np.median(non_null_col_values) if non_null_col_values.size > 0 else np.nan

    def should_impute(i, j):
        condition1 = ui(i) >= Mi(i) and ui(i) >= uj(j)
        condition2 = uj(j) >= ui(i)
        condition3 = Mj(j) >= Mi(i)

        return condition1 or condition2 or condition3

    result_df = np.copy(df)

    for i in range(n_rows):
        for j in range(n_cols):
            if np.isnan(df[i, j]):
                continue

            if should_impute(i, j):
                result_df[i, j] = ui(i) if ui(i) >= uj(j) else uj(j)

    return result_df

In [44]:
import string

def assign_integers(sentence):
    words = sentence.split()
    words_without_punctuation = [''.join(char for char in word if char.isalnum()) for word in words]

    unique_first_letters = sorted(set(word[0].lower() for word in words_without_punctuation))
    letter_to_integer = {letter: i for i, letter in enumerate(unique_first_letters)}

    word_to_integer = {}
    for word in words_without_punctuation:
        first_letter = word[0].lower()
        word_to_integer[word] = letter_to_integer[first_letter]

    return word_to_integer

sentence = 'they know her , jill . but she does not know them .'
result = assign_integers(sentence)
print(result)

IndexError: string index out of range

In [45]:
import string

def assign_integers(sentence):
    words = sentence.split()
    words_without_punctuation = [''.join(char for char in word if char.isalnum()) for word in words]

    unique_first_letters = sorted(set(word[0].lower() for word in words_without_punctuation))
    letter_to_integer = {letter: i for i, letter in enumerate(unique_first_letters)}

    word_to_integer = {}
    for word in words_without_punctuation:
        if word:  # Vérifie si le mot n'est pas vide
            first_letter = word[0].lower()
            word_to_integer[word] = letter_to_integer[first_letter]

    return word_to_integer

sentence = 'they know her , jill . but she does not know them .'
result = assign_integers(sentence)
print(result)

IndexError: string index out of range

In [46]:
arr = np.array([
    [1.2, -0.3, 1.3],
    [1.2, 2., 0.3]
]).astype(np.float64)

arr.shape

(2, 3)

In [47]:
for a in arr:
    print(a)

[ 1.2 -0.3  1.3]
[1.2 2.  0.3]
